##  Fine-tune a pretrained model :  
## NSMC 감성분석 데이터 학습 및 분류기 만들기  

In [1]:
!pip install datasets

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/507.1 kB ? eta -:--:--
   ---------------------------------------- 507.1/507.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/115.3 kB ? eta -:--:--
   ---------------------------------------- 115.3/115.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ---------------------------------------- 166.4/166.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/364.6 kB ? eta -:--:--
   ---------------------------------------- 364.6/364.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/330.3 kB ? eta -:--:--
   ---------------------------------------- 330.3

In [2]:
!pip install sentencepiece
!pip install torch

     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/977.5 kB ? eta -:--:--
     -- ------------------------------------ 61.4/977.5 kB 1.1 MB/s eta 0:00:01
     -------------------------------------  972.8/977.5 kB 8.8 MB/s eta 0:00:01
     -------------------------------------- 977.5/977.5 kB 8.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/5.7 MB 1.7 MB/s eta 0:00:04
     ------ --------------------------------- 1.0/5.7 MB 12.7 MB/s eta 0:00:01
     ------------------------------------- -- 5.4/5.7 MB 48.8 MB/s eta 0:00:01
     ---------------------------------------- 5.7/5.7 MB 45.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     ---------------------------------------- 133.1/133.1 kB ? eta 0:00:00
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     -

In [3]:
!pip install transformers

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 511.8 kB/s eta 0:00:00
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
   ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
   ------------- -------------------------- 92.2/269.5 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 269.5/269.5 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/277.3 kB ? eta -:--:--
   --------------------------------------- 277.3/277.3 kB 16.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 70.6 MB/s eta 0:00:00


In [4]:
# Hugging face에서 데이터 
from datasets import load_dataset
nsmc = load_dataset('nsmc')

c:\ProgramData\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 데이터 확인해 보기
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [6]:
# train data, test_data 분리 (학습 시간 상 2000개만 샘플링)
train_data = nsmc['train'].shuffle(seed=42).select(range(2000))
test_data = nsmc['test'].shuffle(seed=42).select(range(2000))

In [7]:
# train, test 데이터 확인해 보기
train_data
test_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 2000
})

In [8]:
# 감성분석 'bert-base-multilingual-cased' 모델 로드
from transformers import AutoModelForSequenceClassification, AutoTokenizer

MODEL_NAME = 'bert-base-multilingual-cased'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json: 100%|██████████| 625/625 [00:00<?, ?B/s] 
model.safetensors: 100%|██████████| 714M/714M [00:07<00:00, 96.2MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 1.25MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 2.00MB/s]


In [9]:
# train_data['document'][0]
# tokenizer.tokenize()

TypeError: PreTrainedTokenizerFast.tokenize() missing 1 required positional argument: 'text'

In [10]:
# tokenizer.tokenize 테스트
tokenizer(train_data['document'][0])

#TOKEN TYPE IDS : 각 토큰의 문장 IDD
#attention_mask : padding 값 : 1, 0

{'input_ids': [101, 11399, 12225, 119, 9788, 9435, 10739, 71439, 11467, 9485, 38709, 70146, 9788, 9435, 10739, 71439, 11467, 8977, 33305, 11903, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# tokenizer테스트

In [17]:
# data encoding (tokenizing)
# train_encoding
train_encoding = tokenizer(
    train_data['document'],
    return_tensors='pt',
    padding=True,
    truncation=True
)
# test_encoding
test_encoding = tokenizer(
    test_data['document'],
    return_tensors='pt',
    padding=True,
    truncation=True
)

In [18]:
# Encoding된 데이터, 개수 확인하기
train_encoding
print(len(train_encoding['input_ids']), len(test_encoding['input_ids']))

2000 2000


In [19]:
# BERT Special token
print(f"special token ids : {tokenizer.all_special_ids}")
print(f"special token  : {tokenizer.all_special_tokens}")

special token ids : [100, 102, 0, 101, 103]
special token  : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


[Bert fine-tuning: 학습 데이터 만들기]
1. input_ids
2. token_type_ids
3. attention_mask  
----------------------> tokenizer를 활용하여 손쉽게 구성 완료
4. labels

In [20]:
# 데이터 셋 class 구성 (학습 sequence와, label 분류)
import torch
from torch.utils.data import Dataset

class NSMCDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels


    def __getitem__(self, idx):
        data = {key: val[idx] for key, val in self.encodings.items()}
        data['labels'] = torch.tensor(self.labels[idx]).long()

        return data


    def __len__(self):
        return len(self.labels)

In [21]:
# class 실행
train_set = NSMCDataset(train_encoding, train_data['label'])
test_set = NSMCDataset(test_encoding, train_data['label'])



In [22]:
# train셋 확인
train_set[0]

{'input_ids': tensor([  101, 11399, 12225,   119,  9788,  9435, 10739, 71439, 11467,  9485,
         38709, 70146,  9788,  9435, 10739, 71439, 11467,  8977, 33305, 11903,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0,

In [23]:
# test셋 확인
test_set[0]

{'input_ids': tensor([  101, 14796, 27728, 10230,   106,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [30]:
!pip install accelerate -U

In [32]:
!pip install transformers

In [33]:
# Trainer 클래스로 학습하기
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./outputs',
    logging_dir = './logs',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=50,
    #50번째마다 로깅 데이터를 찍겠다
    save_steps=50,
    save_total_limit=2
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [27]:
print(torch.__version__)

2.1.2+cpu


In [26]:
# 학습을 위해 gpu 사용 확인 -> gpu 가능: 'cuda' , gpu 불가능 : 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
#가상환경이라 ...시ㅡ


device(type='cpu')

In [ ]:
# datasets의 load_metric을 활용하여 성능 측정
from datasets import load_metric

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    matrix_1 = load_metric('accuracy')
    matrix_2 = load_metric('f1')

    acc = matrix_1.compute(predictions = preds, references=labels)['accuracy']
    f1 = matrix_2.compute(predictions = preds, references=labels)['f1']

    

    return {f"accuracy" : acc, "f1" : f1}

In [ ]:
# model GPU 사용
model.to(device)

# Trainer 구성
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_set,
    eval_dataset= test_set,
    compute_metrics=compute_metrics
)

In [ ]:
# 모델 학습하기
trainer.eval()

In [ ]:
# 약 25분 소요
#{'loss': 0.6646, 'learning_rate': 4.603174603174603e-05, 'epoch': 0.79}
# {'loss': 0.5363, 'learning_rate': 4.2063492063492065e-05, 'epoch': 1.59}
# {'loss': 0.3994, 'learning_rate': 3.809523809523809e-05, 'epoch': 2.38}
# {'loss': 0.3099, 'learning_rate': 3.412698412698413e-05, 'epoch': 3.17}
# {'loss': 0.188, 'learning_rate': 3.0158730158730158e-05, 'epoch': 3.97}
# {'loss': 0.1881, 'learning_rate': 2.6190476190476192e-05, 'epoch': 4.76}
# {'loss': 0.1174, 'learning_rate': 2.2222222222222223e-05, 'epoch': 5.56}
# {'loss': 0.0919, 'learning_rate': 1.8253968253968254e-05, 'epoch': 6.35}
# {'loss': 0.0817, 'learning_rate': 1.4285714285714285e-05, 'epoch': 7.14}
# {'loss': 0.0565, 'learning_rate': 1.0317460317460318e-05, 'epoch': 7.94}
# {'loss': 0.0391, 'learning_rate': 6.349206349206349e-06, 'epoch': 8.73}
# {'loss': 0.0426, 'learning_rate': 2.3809523809523808e-06, 'epoch': 9.52}
# {'train_runtime': 1618.84, 'train_samples_per_second': 12.355, 'train_steps_per_second': 0.389, 'train_loss': 0.2167289758485461, 'epoch': 10.0}
# TrainOutput(global_step=630, training_loss=0.2167289758485461, metrics={'train_runtime': 1618.84, 'train_samples_per_second': 12.355, 'train_steps_per_second': 0.389, 'train_loss': 0.2167289758485461, 'epoch': 10.0})

In [ ]:
# 모델 평가하기

In [ ]:
# {'eval_loss': 1.198122501373291,
#  'eval_accuracy': 0.766,
#  'eval_f1': 0.7662337662337662,
#  'eval_runtime': 46.6907,
#  'eval_samples_per_second': 42.835,
#  'eval_steps_per_second': 1.349,
#  'epoch': 10.0}

In [ ]:
# ! pip install ipywidgets
# ! conda install -c conda-forge ipympl
# !jupyter labextension list

PyTorch를 활용한 네이티브 방식으로 학습하기  
: TrainingArguments와 Trainer를 활용하지 않고 학습

In [ ]:
from torch.utils.data import DataLoader

# dataloader

In [ ]:
# model load

In [ ]:
# model 확인 

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

# train 메소드
def train(epoch, model, dataloader, optimizer, device):
    return

In [ ]:
# evaluate 메소드
def evaluate(model, dataloader, device):
    return

In [ ]:
# optimizer
from torch.optim import AdamW

In [ ]:
# train

In [ ]:
# evaluate 

In [ ]:
# EVAL | LOSS: 1.2046 ACC: 0.7710 F1: 0.7644